In [ ]:
pip install tensorflow-metal

In [ ]:
print(tf.__version__)


In [1]:
import tensorflow as tf
from tensorflow import keras 

In [2]:
import random
import time
import cv2
import numpy as np
import mss
import mss.tools
from PIL import Image
import pandas as pd
import math
from pynput.mouse import Button, Controller
pd.set_option('display.max_columns', 1000)

In [3]:
# this function takes a screenshot of a particular area of the screen
# it also offers the ability to reduce the resolution by selection only every nth pixel...
# there are certainly better ways to reduce the resolution, but this is where I started
# it also lets you decide if you want to convert to grayscale.  
def screenshot(height, width, x, y, reduction_factor = 1, gray = True):
    
    with mss.mss() as sct:
        # The screen part to capture
        region = {'left': x, 'top': y, 'width': width, 'height': height}

        # Grab the data
        img = sct.grab(region)

        if gray:
            result = cv2.cvtColor(np.array(img), cv2.COLOR_BGRA2GRAY)
        else:
            result = cv2.cvtColor(np.array(img), cv2.COLOR_BGRA2BGR)

        img = result[::reduction_factor, ::reduction_factor]
        return img

In [4]:
# area of screen
top = 153
left = 3
w = 593
h = 470

In [5]:
screenshot(h,w,left,top,reduction_factor=60).shape

(16, 20)

In [6]:
# taking training data, which includes mouse input and screen input in the form of regular screenshots
# vel is not exactly velocity (speed and direction), but it is a combination of speed and position (a contradicktion of heisenberg's principle)
mouse = Controller()
def mouse_motion_ss(tme):
     time.sleep(2)
     ss_interval = 0.1
     loop_length = tme/ss_interval #recording mouse motion/ss for a time duration of tme
     mouse_vel_arr2 = []
     ss_array2 = []
     for i in range(int(loop_length)):
          t1, t2 = mouse.position
#           time.sleep(0.01)
          ss_array2.append(screenshot(h,w,left,top,reduction_factor=60))
#           time.sleep(0.01)
#           t3, t4 = mouse.position
#           speed = math.sqrt((t3-t1)*(t3-t1)+(t4-t2)*(t4-t2))
          mouse_vel_arr2.append([t1,t2])
          time.sleep(ss_interval)
     
     return mouse_vel_arr2, ss_array2

In [7]:
# actaully executing the data recording
mouse_vel2, ss_array2 = mouse_motion_ss(600)

In [8]:
# convert the recorded data into storable form
ss_array2 = np.array(ss_array2)
ss_array2_reshaped = ss_array2.reshape(ss_array2.shape[0],ss_array2.shape[1]*ss_array2.shape[2])
mouse_vel = np.array(mouse_vel2)

In [9]:
# extract the already stored files, concatenate new data in them, and re-store them on disk
temp = np.loadtxt('mouse_vel2.txt', dtype = 'float')
temp2 = np.concatenate((temp,mouse_vel2))
np.savetxt('mouse_vel2.txt', temp2, fmt = '%f')
temp3 = np.loadtxt('ss_array2.txt', dtype = 'int')
temp4 = np.concatenate((temp3, ss_array2_reshaped))
np.savetxt('ss_array2.txt', temp4, fmt = '%d')

In [38]:
# np.savetxt('ss_array2.txt', ss_array2_reshaped, fmt = '%d')
# np.savetxt('mouse_vel2.txt', mouse_vel2, fmt = '%f')

In [ ]:
%pwd

In [ ]:
reconstructed_model = keras.models.load_model("model1")

In [ ]:
file = open('ss_array.txt', 'a')
file.write(ss_array_reshaped)
file2 = open('mouse_vel.txt', 'a')
file2.write(mouse_vel)

In [ ]:
ans = []
temp2 = ''
for i in range (120):
     temp2 = temp2+temp[i]

In [ ]:
file

In [ ]:
mouse_vel[0]

In [17]:
# from pynput import keyboard
# # The event listener will be running in this block
# with keyboard.Events() as events:
#     # Block at most one second
#     event = events.get(10)

mouse = Controller()
def move_mouse_random (h,w,left,top):
     for j in range(100):
          x = random.randint(left, left+w)
          y = random.randint(top, top+h)
          xc, yc = mouse.position
          smoothness = 30
          for i in range (smoothness):
               mouse.position = ((xc+((x-xc)*i)/smoothness),(yc+((y-yc)*i)/smoothness))
               time.sleep(1/smoothness)
          if j == 0: mouse.click(Button.left,1)
          print(x,y)
          time.sleep(0.5)
     return

In [20]:
move_mouse_random(h,w,left,top)

52 430
576 401
404 337
315 472
120 182
452 401
490 564
414 400
591 273
441 433
335 154
551 298
241 223
305 259
93 233
582 251
534 510
173 280
4 173
96 380
342 502
410 415
153 269
399 568
527 341
131 253
5 382
440 361
118 334
211 189
93 619
181 284
453 593
497 538
252 300
43 336
121 582
127 361
265 566
417 443
377 353
552 508
119 241
201 345
570 339
12 567
271 610
527 411
482 493
18 181
403 574
419 281
368 276
520 547
546 545
63 548
329 506
200 215
40 380
506 584
389 363
95 374
468 541
371 365
261 320
460 594
33 581
469 271
89 209
155 450
415 219
328 196
274 408
71 320
477 337
336 419
180 278
210 263
348 270
554 240
441 550
274 478
509 318
552 572
431 553
329 541
254 559
207 566
343 583
128 199
207 283
410 549
380 435
439 554
395 196
122 354
443 473
310 288
245 222
585 189


In [74]:
temp_ss = np.array(screenshot(h,w,left,top,reduction_factor=30))
temp_ss = np.reshape(temp_ss, (temp_ss.shape[0]*temp_ss.shape[1],))
temp_ss = temp_ss/255

In [103]:
np.array([temp_ss])[[0]]

array([[1.        , 1.        , 1.        , ..., 1.        , 1.        ,
        0.84705882]])

In [99]:
temp_ss2 = []
temp_ss2.append(temp_ss)
temp_ss2 = np.array(temp_ss2)

In [100]:
temp_ss2[[0]]

array([[1.        , 1.        , 1.        , ..., 1.        , 1.        ,
        0.84705882]])

In [105]:
np.argmax(model1.predict(np.array([temp_ss])[[0]]))*100

0

In [23]:
mouse = Controller()
def input_output (tme):
    # initialize mouse position
    init_ss = np.array(screenshot(h,w,left,top,reduction_factor=60))
    init_ss = np.reshape(init_ss, (init_ss.shape[0]*init_ss.shape[1],))
    init_ss = init_ss/255
    mouse.position = (np.argmax(model1.predict(np.array([init_ss])[[0]]))*100, np.argmax(model2.predict(np.array([init_ss])[[0]]))*100)
    mouse.click(Button.left,1)
    for i in range(tme*10):
        ss = np.array(screenshot(h,w,left,top,reduction_factor=60))
        ss = np.reshape(ss, (ss.shape[0]*ss.shape[1],))
        ss = ss/255
#         speed = np.argmax(model3.predict(np.array([init_ss])[[0]]))*100
        smoothness = 10
        
        x = np.argmax(model1.predict(np.array([ss])[[0]]))*100
        y = np.argmax(model2.predict(np.array([ss])[[0]]))*100
        
        for j in range(smoothness):
            frame_duration = 0.1/smoothness
            xc, yc = mouse.position
            mouse.position = ((xc+((x-xc)*(j+1))/smoothness),(yc+((y-yc)*(j+1))/smoothness))
            time.sleep(frame_duration)
        
        

In [34]:
input_output(60)

In [21]:
# These are the dimensions of the window that I'm going to use
window_x = 0.0
window_y = 150 # The tool bar in chrome is 100 pixels (or so) tall

# Too high resolution and the dataset gets too big.  This is the size of the window to create
width = 600 
height = 600

# size of the game window (the playable area)
window_width = width 
window_height = height - window_y

# coordinates of the center of the game windows
window_center_x = window_width/2
window_center_y = window_y + window_height/2

#######################
# comment these out if you want to consider the whole window
# this is so that you can easily consider less data if you want to
size_of_window = 200.0
window_x = window_center_x - size_of_window/2
window_y = window_center_y - size_of_window/2
window_width = size_of_window
window_height = size_of_window
#######################

# Location of the score that is displayed on the screen
# This will change if you change the dimensions of the window that you create.  
# YOu can easily use mouse.position() to identify the new coordinates
score_x = 83
score_y = 630
score_width = 91
score_height = 25

# position of the start button on the screen with these window settings
# Note that the shape of the start button changes depending on whether it's a brand new game
# or if you've just died
start_button_position_x = 314
start_button_position_y = 496

# instantiate mouse
mouse = Mouse()

# open slither.io
driver = open_and_size_browser_window(width = width, height = height)

# make some data
generate_data(delete_file = True, verbose = True)

NameError: name 'Mouse' is not defined

In [ ]:
# from pynput import keyboard

# # The event listener will be running in this block
# with keyboard.Events() as events:
#     # Block at most one second
#     event = events.get(10)
#     if event is None:
#         print('You did not press a key within one second')
#     else:
#         print('Received event {}'.format(event))

In [ ]:
# from pynput.keyboard import Key,Controller
# keyboard = Controller()
# keyboard.press()

In [ ]:
# region = {'left': left, 'top': top, 'width': w, 'height': h}
# screenshot = mss.mss().grab(region)
# img = Image.frombytes("RGB", screenshot.size, screenshot.bgra, "raw", "BGRX")  # Convert to PIL.Image
# img.show()

In [ ]:
# def ss_take(tme, h, w, left, top):
     #      time.sleep(2.01)
#      ss_interval = 0.1
#      ss_array = []
#      loop_length = tme/ss_interval #recording mouse motion for a time duration of tme
#      for i in range(int(loop_length)):
#           ss_array.append(screenshot(h,w,left,top))
#           time.sleep(ss_interval)
#      return ss_array

19800

In [10]:
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import pandas as pd

#loading files
y = np.loadtxt('mouse_vel2.txt', dtype = 'float')
X = np.loadtxt('ss_array2.txt', dtype = 'int')

# three models
y1 = []
y2 = []
# y3 = []
for i in range(len(y)):
  y1.append(int(y[i][0]/100))
  y2.append(int(y[i][1]/100))
#   y3.append(int(y[i][2]/100))

# preprocessing X
X_scaled = X/255
X_list = []
for i in range (len(X)):
  tmp = X_scaled[i].tolist()
  X_list.append(tmp)

In [11]:
from sklearn.model_selection import train_test_split
y_train1, y_test1, X_train1, X_test1 = train_test_split(y1, X_list, test_size = 0.2) 
model1 = keras.Sequential([
    keras.layers.Dense(1000, input_shape=(320,), activation='relu'),
    keras.layers.Dense(1000, activation = 'relu'),
    keras.layers.Dense(500, activation = 'relu'),
    keras.layers.Dense(500, activation = 'relu'),
    keras.layers.Dense(100, activation = 'relu'),
    keras.layers.Dense(50, activation = 'relu'),
    keras.layers.Dense(20, activation = 'relu'),
    keras.layers.Dense(7, activation='sigmoid')
])

model1.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model1.fit(X_train1, y_train1, epochs=15)

2022-06-16 19:03:06.542397: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-06-16 19:03:06.543266: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Metal device set to: Apple M1


2022-06-16 19:03:19.324728: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:176] None of the MLIR Optimization Passes are enabled (registered 2)
2022-06-16 19:03:19.330755: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


Epoch 1/15


2022-06-16 19:03:19.640698: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


795/795 [==============================] - 7s 8ms/step - loss: 1.6162 - accuracy: 0.2480
Epoch 2/15
795/795 [==============================] - 7s 9ms/step - loss: 1.5672 - accuracy: 0.2816
Epoch 3/15
795/795 [==============================] - 6s 8ms/step - loss: 1.5176 - accuracy: 0.3336
Epoch 4/15
795/795 [==============================] - 7s 8ms/step - loss: 1.4375 - accuracy: 0.3848
Epoch 5/15
795/795 [==============================] - 7s 8ms/step - loss: 1.3542 - accuracy: 0.4298
Epoch 6/15
795/795 [==============================] - 6s 8ms/step - loss: 1.2608 - accuracy: 0.4711
Epoch 7/15
795/795 [==============================] - 7s 9ms/step - loss: 1.1645 - accuracy: 0.5141
Epoch 8/15
795/795 [==============================] - 6s 8ms/step - loss: 1.0696 - accuracy: 0.5582
Epoch 9/15
795/795 [==============================] - 6s 8ms/step - loss: 0.9742 - accuracy: 0.5994
Epoch 10/15
795/795 [==============================] - 6s 8ms/step - loss: 0.8854 - accuracy: 0.6420
Epoch 11/1

In [22]:
from sklearn.model_selection import train_test_split
y_train2, y_test2, X_train2, X_test2 = train_test_split(y2, X_list, test_size = 0.2) 
from tensorflow.keras.optimizers import SGD
model2 = keras.Sequential([
    keras.layers.Dense(1000, input_shape=(320,), activation='relu'),
    keras.layers.Dense(1000, activation = 'relu'),
    keras.layers.Dense(500, activation = 'relu'),
    keras.layers.Dense(500, activation = 'relu'),
    keras.layers.Dense(100, activation = 'relu'),
    keras.layers.Dense(50, activation = 'relu'),
    keras.layers.Dense(20, activation = 'relu'),
    keras.layers.Dense(7, activation='sigmoid')
])
opt = SGD(lr=0.01, momentum=0.9, clipvalue=0.5)
model2.compile(optimizer=opt,
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model2.fit(X_train2, y_train2, epochs=15)

/Users/shashankkatiyar/miniforge3/envs/tensorflow/lib/python3.9/site-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(


Epoch 1/15


2022-06-16 19:28:00.299822: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


795/795 [==============================] - 7s 7ms/step - loss: 1.5446 - accuracy: 0.2919
Epoch 2/15
795/795 [==============================] - 6s 7ms/step - loss: 1.4628 - accuracy: 0.3390
Epoch 3/15
795/795 [==============================] - 6s 7ms/step - loss: 1.3860 - accuracy: 0.3942
Epoch 4/15
795/795 [==============================] - 6s 8ms/step - loss: 1.3180 - accuracy: 0.4351
Epoch 5/15
795/795 [==============================] - 6s 7ms/step - loss: 1.2419 - accuracy: 0.4807
Epoch 6/15
795/795 [==============================] - 7s 8ms/step - loss: 1.1669 - accuracy: 0.5200
Epoch 7/15
795/795 [==============================] - 6s 8ms/step - loss: 1.0754 - accuracy: 0.5653
Epoch 8/15
795/795 [==============================] - 6s 8ms/step - loss: 0.9874 - accuracy: 0.6029
Epoch 9/15
795/795 [==============================] - 6s 8ms/step - loss: 0.9058 - accuracy: 0.6412
Epoch 10/15
795/795 [==============================] - 6s 7ms/step - loss: 0.8032 - accuracy: 0.6869
Epoch 11/1

In [15]:
model1.evaluate(X_test2,y_test2)

2022-06-16 19:21:42.495416: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


199/199 [==============================] - 2s 6ms/step - loss: 6.5205 - accuracy: 0.1802


[6.520516872406006, 0.18018867075443268]

In [7]:
from sklearn.model_selection import train_test_split
y_train3, y_test3, X_train3, X_test3 = train_test_split(y3, X_list, test_size = 0.2) 
model3 = keras.Sequential([
    keras.layers.Dense(1000, input_shape=(1280,), activation='relu'),
    keras.layers.Dense(1000, activation = 'relu'),
    keras.layers.Dense(500, activation = 'relu'),
    keras.layers.Dense(500, activation = 'relu'),
    keras.layers.Dense(100, activation = 'relu'),
    keras.layers.Dense(50, activation = 'relu'),
    keras.layers.Dense(20, activation = 'relu'),
    keras.layers.Dense(7, activation='sigmoid')
])

model3.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model3.fit(X_train3, y_train3, epochs=15)

Epoch 1/15
 7/27 [======>.......................] - ETA: 0s - loss: 0.5865 - accuracy: 0.9732

2022-06-16 01:04:44.416543: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


27/27 [==============================] - 0s 8ms/step - loss: 0.2430 - accuracy: 0.9907
Epoch 2/15
27/27 [==============================] - 0s 8ms/step - loss: 0.0358 - accuracy: 0.9965
Epoch 3/15
27/27 [==============================] - 0s 8ms/step - loss: 0.0244 - accuracy: 0.9965
Epoch 4/15
27/27 [==============================] - 0s 8ms/step - loss: 0.0233 - accuracy: 0.9965
Epoch 5/15
27/27 [==============================] - 0s 8ms/step - loss: 0.0184 - accuracy: 0.9965
Epoch 6/15
27/27 [==============================] - 0s 9ms/step - loss: 0.0094 - accuracy: 0.9965
Epoch 7/15
27/27 [==============================] - 0s 8ms/step - loss: 0.0065 - accuracy: 0.9965
Epoch 8/15
27/27 [==============================] - 0s 8ms/step - loss: 0.0061 - accuracy: 0.9965
Epoch 9/15
27/27 [==============================] - 0s 9ms/step - loss: 0.0034 - accuracy: 0.9988
Epoch 10/15
27/27 [==============================] - 0s 8ms/step - loss: 0.4192 - accuracy: 0.9595
Epoch 11/15
27/27 [===========

In [25]:
X.shape

(1080, 1280)

In [33]:
X[0]

array([21, 31, 93, ..., 26, 18, 20])

In [34]:
model1.test_on_batch(X)

2022-06-15 23:47:51.680620: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


[0.6289454102516174, 0.6724537014961243]

In [63]:
model1.predict(temp[[0]])

array([[0.68691725, 0.8137793 , 0.77220684, 0.9091423 , 0.92654246,
        0.05963868, 0.0027196 ]], dtype=float32)

In [49]:
temp = np.array(X_test1)

In [52]:
temp[0]

array([0.11372549, 0.0745098 , 0.09019608, ..., 0.04313725, 0.08627451,
       0.12941176])

In [62]:
temp[[0]]

array([[0.11372549, 0.0745098 , 0.09019608, ..., 0.04313725, 0.08627451,
        0.12941176]])